In [1]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch()

In [21]:
import os
import json
import pprint

## Table Create

In [70]:
# db 이름 설정
INDEX_NAME = "news_wiki_index_update"

# db 셋팅
INDEX_SETTINGS = {
"settings" : {
    "index":{
    "analysis":{
        "analyzer":{
        "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

        }
        }
    }
    }
},
"mappings": {

    "properties" : {
        
        "admin_id" : {
        "type" : "keyword",
        },
        
        "category" : {
        "type" : "keyword",
        },
        
        "date" : {
        "type" : "date"
        },
        
        "title" : {
        "type" : "keyword",
        },
        
        "article" : {
        "type" : "text"
        },
        
        "context" : {
        "type" : "text",
        "analyzer": "korean",
        "search_analyzer": "korean"
        }
        
    }

}
}


In [71]:
def create_index(INDEX_NAME, INDEX_SETTINGS):
    if es.indices.exists(INDEX_NAME):
        es.indices.delete(index=INDEX_NAME)
    es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


In [72]:
create_index(INDEX_NAME, INDEX_SETTINGS)

<ipython-input-71-c9523f9305a3>:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-71-c9523f9305a3>:4: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


## Table Insert

In [16]:
with open(os.path.join("../data/", "mbn_news_wiki_update.json"), "r", encoding="utf-8") as f:
    wiki = json.load(f)

In [42]:
pprint.pprint(len(wiki['0']['article']))

1240


In [64]:
len(wiki.values())

842904

In [69]:
# 데이터 elastic search 입력 형식으로 변환
wikis =[]
for doc in wiki.values():
    temp = {
    "_index": INDEX_NAME,
    "_id" : doc['news_id'],
    "_source": {
        "admin_id":doc['admin_id'],
        "category":doc['category'],
        "date":doc['date'],
        "title":doc['title'],
        "article":doc['article'],
        "context": doc['context']
    }
}
    wikis.append(temp)

7


In [73]:
len(wikis)

842904

In [74]:
# 데이터 입력
try:
    for idx in range(1000,len(wikis), 1000):
        response = helpers.bulk(es, wikis[idx-1000:idx])
        # print ("\nRESPONSE:", response)
    response = helpers.bulk(es, wikis[842000:len(wikis)])
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)


ERROR: ('1 document(s) failed to index.', [{'index': {'_index': 'news_wiki_index_update', '_type': '_doc', '_id': '3567855', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'json_parse_exception', 'reason': "Non-standard token 'NaN': enable JsonParser.Feature.ALLOW_NON_NUMERIC_NUMBERS to allow\n at [Source: (ByteArrayInputStream); line: 1, column: 77]"}}, 'data': {'admin_id': 'AIPaperboy', 'category': '정치', 'date': '2018-06-25', 'title': nan, 'article': '【 앵커멘트 】<br /><br />  문재인 대통령이 고 김종필 전 국무총리의 빈소를 방문하지 않기로 결정했습니다. <br /><br />  대신, 국민훈장 무궁화장을 추서하면서 논란을 최소화하는 정치적 판단을 한 것으로 보입니다. <br /><br />  최중락 기자가 보도합니다. <br /><br /><br /><br /><br /><br />【 기자 】<br /><br />  문재인 대통령은 고심 끝에 고 김종필 전 총리에게 국민훈장 무궁화장을 추서하되 직접 조문은 하지 않기로 했습니다.<br /><br /><br /><br />  청와대 김의겸 대변인은 "김부겸 행정안전부 장관이 국민훈장 무궁화장을 추서할 예정"이라고 밝혔습니다. <br /><br /><br /><br />  "문 대통령은 유족에게 예우를 갖춰 애도를 표하라"고 김 장관에게 지시했다고 전했습니다. <br /><br /><br /><br />  "대통령의 조문은 이것

In [2]:
def insert(body):
    return es.index(index=index, doc_type=doc_type, body=body)


'''
index = 'news'
doc_type = 'daum'
data = {
    'date' : '20200811',
    'category' : 'society',
    'title' : '유령'은 어느 행성에서 지구로 왔는가?'
    'content' : '크립톤 행성'
}

ir = insert(data)
'''

## Table Read

In [75]:
res = es.get(index = "news_wiki_index_update", id='3105313')
res

{'_index': 'news_wiki_index_update',
 '_type': '_doc',
 '_id': '3105313',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'admin_id': 'AIPaperboy',
  'category': '정치',
  'date': '2017-01-01',
  'title': "'붉은 닭의 해' 정유년 첫 날, 대통령은?…靑관저서 참모들과 '떡국 조찬'",
  'article': '\'붉은 닭의 해\' 정유년 첫 날, 대통령은?…靑관저서 참모들과 \'떡국 조찬\'<br /><br /><br /><br /><!------------ PHOTO_POS_0 ------------><br /><br />박근혜 대통령이 새해 첫날인 1일 청와대 관저에서 참모들과 \'떡국 조찬\'을 합니다.<br /><br /><br /><br />2013년 취임 후 매년 1월 1일에는 국립현충원을 참배하고 국무위원 및 청와대 참모들과 떡국으로 아침을 먹었으나, 올해는 참배 없이 참모들과만 아침 식사를 같이하기로 했습니다.<br /><br /><br /><br />지난달 9일 국회의 탄핵소추안 가결로 직무가 정지된 상황임을 고려한 것입니다.<br /><br /><br /><br />박 대통령이 새해 첫날 내던 신년사와 국군 장병에 대한 격려 메시지도 올해는 황교안 대통령 권한대행 국무총리가 대신했습니다.<br /><br /><br /><br />이날 떡국 조찬 자리는 박 대통령이 최근 청와대 관저를 찾아온 참모들에게 "새해에 떡국이라도 드셔야 하지 않겠느냐"고 말한데 이어 참모들도 박 대통령에게 "혼자 드시지 마라"고 건의하면서 만들어진 것으로 알려졌습니다.<br /><br /><br /><br />새해 첫 식사자리인 만큼 건강 기원 등의 덕담이 서로 오가는 자리가 될 것으로 보입니다.<br /><br /><br /><br />다만 헌법재판소의

In [77]:
res['_source']['article']

'\'붉은 닭의 해\' 정유년 첫 날, 대통령은?…靑관저서 참모들과 \'떡국 조찬\'<br /><br /><br /><br /><!------------ PHOTO_POS_0 ------------><br /><br />박근혜 대통령이 새해 첫날인 1일 청와대 관저에서 참모들과 \'떡국 조찬\'을 합니다.<br /><br /><br /><br />2013년 취임 후 매년 1월 1일에는 국립현충원을 참배하고 국무위원 및 청와대 참모들과 떡국으로 아침을 먹었으나, 올해는 참배 없이 참모들과만 아침 식사를 같이하기로 했습니다.<br /><br /><br /><br />지난달 9일 국회의 탄핵소추안 가결로 직무가 정지된 상황임을 고려한 것입니다.<br /><br /><br /><br />박 대통령이 새해 첫날 내던 신년사와 국군 장병에 대한 격려 메시지도 올해는 황교안 대통령 권한대행 국무총리가 대신했습니다.<br /><br /><br /><br />이날 떡국 조찬 자리는 박 대통령이 최근 청와대 관저를 찾아온 참모들에게 "새해에 떡국이라도 드셔야 하지 않겠느냐"고 말한데 이어 참모들도 박 대통령에게 "혼자 드시지 마라"고 건의하면서 만들어진 것으로 알려졌습니다.<br /><br /><br /><br />새해 첫 식사자리인 만큼 건강 기원 등의 덕담이 서로 오가는 자리가 될 것으로 보입니다.<br /><br /><br /><br />다만 헌법재판소의 탄핵심판 절차가 본격화되고 박영수 특별검사팀의 수사도 속도를 내고 있어 이전처럼 편안한 분위기가 되기는 어려울 것으로 전망됩니다. <br /><br /><br /><br />특히 박 대통령의 퇴진을 촉구하는 여론이 계속되면서 박 대통령측 일각에서는 법적 절차 진행이 여론의 영향을 받는 게 아니냐는 우려도 있습니다.<br /><br /><br /><br />탄핵안 가결 직후와 달리 박 대통령은 최근 담담하고 차분한 분위기를 보이고 있지만 이런 여론 환경 등을 고려할 때 심기가 편할 수 없다는 게 박 대통령 측 인사들의 전언

### 카테고리 지정, 날짜 범위 지정, 쿼리 지정

In [78]:
def search(index, data=None):
    if data is None:
        data = {"match_all" : {}}
    else:
        data = {"match" : data}
    body = {"query" : data}
    res = es.search(index=index, body=body)
    
'''
sr = search(index, {'category': 'society'})
pprint.pprint(sr)
'''

"\nsr = search(index, {'category': 'society'})\npprint.pprint(sr)\n"

### 정렬하기, 호출 데이터 갯수 지정

In [98]:
body = {
  "query": {
    "bool": {
      "filter": [
        { "term": { "category": "정치"}},
        { "term": { "date": "2017-01-01" }}
      ]
    }    
    }
  }

query = "문재인이 시행한 정책은?"
es.search(index = INDEX_NAME, q = query, body = body)

<ipython-input-98-3b300d5cebcc>:13: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index = INDEX_NAME, q = query, body = body)


{'took': 14,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 14.62258,
  'hits': [{'_index': 'news_wiki_index_update',
    '_type': '_doc',
    '_id': '3492632',
    '_score': 14.62258,
    '_source': {'admin_id': 'AIPaperboy',
     'category': '경제',
     'date': '2018-03-30',
     'title': '의협, 4월 휴진 등 집단행동 예고…"문재인 케어와 전쟁"<br />',
     'article': '<br />의사 직능단체인 대한의사협회가 정부의 건강보험 보장성 강화 정책인 \'문재인 케어\'와의 전쟁을 선포했다.<br />의협은 4월 말 의료계가 동참하는 집단휴진, 총궐기대회 등 집단행동에 나설 방침이다. 집단 행동 날짜는 4월 22일, 27일, 29일 등이다. 최대집 제40대 대한의사협회 회장 당선인은 30일 긴급 기자회견을 열고 "의료(행위)를 멈춰서라도 \'문재인 케어\'를 강력히 저지하겠다"며 이같이 밝혔다. <br /> 의협에서 거론하는 집단행동은 전국의사 총궐기대회와 같은 대규모 시위, 전일 또는 반일 집단휴진 등이다. 아직 집단행동 방식이나 일정 등 구체적인 계획은 확정되지 않았다. 의협이 집단행동을 예고하고 나선것은 정부가 문재인 케어의 본격적인 시행을 알리는 \'상복부 초음파 건강보험 적용\' 정책을 애초 예고한대로 4월 1일부터 시행한 데 따른 것이다. 당초 의협은 복지부에 의료계와 협의 없이 강행된 상복부 초음파 급여화 고시를 철회하라고 요구한 바 있다. 의협은 시행 연기 요구가 받아들여지지 않자 정부와의 실무협상을 

In [103]:
body = {
  "query": {
      "range": {
      "date": {
        "gt": "2016-01-01",
        "lt": "2016-01-01||+3M"
      }
    }
  }
  }

query = "문재인이 시행한 정책은?"
es.search(index = INDEX_NAME, q = query, body = body)

<ipython-input-103-b02abc008e71>:13: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index = INDEX_NAME, q = query, body = body)


{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 14.62258,
  'hits': [{'_index': 'news_wiki_index_update',
    '_type': '_doc',
    '_id': '3492632',
    '_score': 14.62258,
    '_source': {'admin_id': 'AIPaperboy',
     'category': '경제',
     'date': '2018-03-30',
     'title': '의협, 4월 휴진 등 집단행동 예고…"문재인 케어와 전쟁"<br />',
     'article': '<br />의사 직능단체인 대한의사협회가 정부의 건강보험 보장성 강화 정책인 \'문재인 케어\'와의 전쟁을 선포했다.<br />의협은 4월 말 의료계가 동참하는 집단휴진, 총궐기대회 등 집단행동에 나설 방침이다. 집단 행동 날짜는 4월 22일, 27일, 29일 등이다. 최대집 제40대 대한의사협회 회장 당선인은 30일 긴급 기자회견을 열고 "의료(행위)를 멈춰서라도 \'문재인 케어\'를 강력히 저지하겠다"며 이같이 밝혔다. <br /> 의협에서 거론하는 집단행동은 전국의사 총궐기대회와 같은 대규모 시위, 전일 또는 반일 집단휴진 등이다. 아직 집단행동 방식이나 일정 등 구체적인 계획은 확정되지 않았다. 의협이 집단행동을 예고하고 나선것은 정부가 문재인 케어의 본격적인 시행을 알리는 \'상복부 초음파 건강보험 적용\' 정책을 애초 예고한대로 4월 1일부터 시행한 데 따른 것이다. 당초 의협은 복지부에 의료계와 협의 없이 강행된 상복부 초음파 급여화 고시를 철회하라고 요구한 바 있다. 의협은 시행 연기 요구가 받아들여지지 않자 정부와의 실무협상을 

## Table Update

In [ ]:
def update(id, doc):
    body = {'doc' : doc}
    res = es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res

ir = insert(data)
ur = update(ir['_id'], {newspaper:'sbs'})
pprint.pprint(ur)

## Table Delete

In [ ]:
def delete_index(index):
    if not es.indices.exists(index=index):
        return es.indices.delete(index=index)

In [ ]:
def delete_by_id(index, id):
    return es.delete(index, id=id)